In [1]:
#Imports
# import json
# import os

# import requests
# from tenacity import retry, wait_random_exponential, stop_after_attempt
# from termcolor import colored

import streamlit as st
import inspect
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain, ConversationalRetrievalChain, LLMChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory, ConversationKGMemory
from langchain.schema import StrOutputParser, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)

from operator import itemgetter

from ftstringprompts import *


from dotenv import load_dotenv
# from EduFlowSinglePrompts import *


load_dotenv("../.venv/.env")


'{ayyaba}'

#### inputs

In [2]:
key_idea = """On planet Earth, life exists in hostile and extreme environments, and the organisms that survive there are termed extremophiles."""
key_idea_description = """Extremophiles are organisms that thrive in extreme conditions on Earth, such as high temperatures, acidity, alkalinity, or pressure. These environments include locations like deep-sea hydrothermal vents, acidic hot springs, or subglacial lakes in Antarctica. The adaptability of these organisms expands our understanding of life's limits and opens up possibilities for life in extreme conditions beyond Earth."""
theory = """Extremophiles and Their Environments
Definition: Extremophiles are primarily unicellular microbes, including bacteria and archaea, that live in harsh environments​​.
Habitats: These environments include deep-sea hydrothermal vents with high pressure and temperatures (up to 122 °C), volcanic hot springs, deep underground in mines, and in bodies of water that are highly acidic, alkaline, extremely salty, or even radioactive​​.
The Significance of Extremophiles
Adaptation and Survival: Extremophiles have adapted to survive in conditions that are generally detrimental to most life forms. Their existence expands our understanding of the limits and versatility of life.
Implications for Extraterrestrial Life: The study of extremophiles has significant implications for astrobiology. The discovery of diverse microbial communities in extreme environments like subglacial lakes under the Antarctic ice sheet suggests that life might exist in similar extreme conditions elsewhere in the universe, such as on ice-covered moons like Europa​​.
Europa: A Candidate for Extraterrestrial Life
Characteristics of Europa: Europa, a moon of Jupiter, is a primary candidate for extraterrestrial life exploration. It has an ice-covered surface, many kilometers thick, with a possible sub-surface ocean of salty water. The surface of Europa stretches and relaxes due to tidal movements caused by its elliptical orbit around Jupiter, generating heat that could keep a sub-surface ocean in liquid state​​.
Potential Habitability: Europa is considered a promising location for life beyond Earth due to its thin ice shell, liquid ocean, and contact with a geologically active rocky core. The moon has water and the right chemical elements, along with a potentially stable environment​​.
Broader Understanding
Cellular Requirements for Life: For life to exist, certain conditions are necessary, including the availability of an energy source, liquid water for biochemical reactions, and essential chemical building blocks like carbon, oxygen, nitrogen, and hydrogen. Life can thrive in stable environmental conditions within a specific range of pressure, temperature, light intensity, pH, and salinity​​.
Challenging Traditional Concepts: Extremophiles challenge the traditional concept of habitable environments, indicating that life can exist in conditions previously thought to be uninhabitable. This expands our understanding of possible habitats for life, both on Earth and potentially in extraterrestrial settings.
This comprehensive overview illustrates the remarkable adaptability of life and opens up new possibilities for understanding life's potential existence in extreme environments, both on Earth and beyond."""
student_year_level = "Year 11"
subject = "Biology"
question_difficulty = "moderate"
question_examples_easy = ["What is an extremophile? An extremophile is an organism that thrives in extreme environmental conditions that are typically detrimental to most life forms.", "Give one example of an extreme environment on Earth. One example is deep-sea hydrothermal vents.", "What is unique about Europa, one of Jupiter's moons? Europa has an ice-covered surface and is thought to have a sub-surface ocean of salty water."]
question_examples_moderate = ["How do extremophiles survive in high temperatures? Extremophiles in high temperatures have specialized proteins and enzymes that remain stable and functional at elevated temperatures.", "Why is the study of extremophiles important? Studying extremophiles helps understand the adaptability and limits of life, and it may provide insights into the possibility of life on other planets or moons.", "Why is Europa considered a likely place to find life beyond Earth? Europa is considered likely because of its ice shell, liquid ocean, and geologically active core, which provide conditions potentially suitable for life."]
question_examples_hard = ["Explain how extremophiles challenge the traditional concept of habitable environments. Extremophiles challenge the traditional concept by surviving in environments previously thought to be uninhabitable, expanding our understanding of where life can exist", "Discuss the potential implications of finding life in subglacial lakes in Antarctica for the search for extraterrestrial life. The discovery in subglacial lakes suggests that similar extreme, ice-covered environments in the solar system, such as on Europa, could also harbor life.", "How does the study of extremophiles contribute to our understanding of the potential for life on planets and moons outside the 'Goldilocks Zone'? The study of extremophiles indicates that life can exist in conditions not reliant on a star's warmth, suggesting that planets and moons outside the 'Goldilocks Zone' could also support life."]
subject = 'biology'
question_examples = question_examples_moderate
other_system_instructions = ""
student_interests = ["gaming", "politics", "philosophy"]
student_career_goals = ["lawyer", "doctor", "engineer"]

In [3]:
def create_education_dict(key_idea, key_idea_description, theory, student_year_level, subject, question_difficulty, question_examples_moderate,  student_interests, student_career_goals, other_system_instructions):
    education_dict = {
        "key_idea": key_idea,
        "key_idea_description": key_idea_description,
        "theory": theory,
        "student_year_level": student_year_level,
        "subject": subject,
        "question_difficulty": question_difficulty,
        "question_examples": question_examples_moderate,
        "student_career_goals": student_career_goals,
        "student_interests": student_interests,
        "other_system_instructions": other_system_instructions
        }
    return education_dict

data = create_education_dict(key_idea, key_idea_description, theory, student_year_level, subject, question_difficulty, question_examples_moderate,  student_interests, student_career_goals, other_system_instructions)

#### Test code

In [4]:
model = "gpt-3.5-turbo-1106"

llm = ChatOpenAI(temperature=0.9,model=model)
prompt, input_variables, human = generate_question()
sys_message = SystemMessagePromptTemplate.from_template(prompt)
human_message = HumanMessagePromptTemplate.from_template(human)
# prompt = generate_question_template.format_prompt(**data)

chat_template = ChatPromptTemplate.from_messages([sys_message, human_message])

llm(chat_template.format_messages(**data))


AIMessage(content='--BEGIN RESPONSE--\n\nExplain the significance of extremophiles in astrobiology, and how their existence expands our understanding of the limits and adaptability of life. Discuss their potential implications for the possibility of life on other planets or moons, such as Europa, in the context of habitable environments.\n\n--END RESPONSE--')

In [5]:
#Sequence 1
model = ChatOpenAI(temperature = temp, model_name= model)

#prompt templates
generate_question_prompt_messages = generate_question()
answer_prompt_messages = answer_question()
hallucination_check_prompt_messages = check_generated_answer_hallucination()

#message objects
final_generate_prompt_question = ChatPromptTemplate.from_messages(generate_question_prompt_messages)
final_answer_prompt = ChatPromptTemplate.from_messages(answer_prompt_messages)
hallucination_check_prompt_messages = ChatPromptTemplate.from_messages(hallucination_check_prompt_messages)

question_stage = final_generate_prompt_question | model


NameError: name 'temp' is not defined